# Laboratorio 8

## Integrantes

### Sergio Orellana - 221122

### Andre Marroquin - 22266

### Rodrigo Mansilla - 22611

# Link del repositorio

https://github.com/mar22266/LABORATORIOS-IA.git


# Task 1 Teoría


1.  Investigar el algoritmo AC-3 y su relación con el algoritmo de backtracking search

        R// El algoritmo AC-3 (Arc Consistency Algorithm 3) es un método utilizado en problemas de satisfacción de restricciones (CSP) para lograr la consistencia de arcos. Su función principal es reducir los dominios de las variables eliminando valores que no satisfacen las restricciones binarias entre pares de variables. Esto se realiza iterativamente hasta que no se puedan hacer más reducciones o se detecte que algún dominio queda vacío, indicando que el CSP no tiene solución.

        Por otro lado, el backtracking search es una técnica de búsqueda en profundidad que asigna valores a las variables de manera tentativa y retrocede backtracks cuando se encuentra una inconsistencia, explorando así todas las posibles combinaciones hasta encontrar una solución o determinar que no existe.

        La relación entre AC-3 y el backtracking search esta en que AC-3 puede usarse como un paso previo al backtracking para reducir los dominios de las variables, facilitando así la búsqueda. Al aplicar AC-3 antes de iniciar el backtracking, se eliminan valores inconsistentes, lo que puede disminuir significativamente el espacio de búsqueda y mejorar la eficiencia del proceso de

2.  Defina en sus propias palabras el término “Arc Consistency”

        R// La "consistencia de arcos" es una propiedad que aparece en problemas de satisfacción de restricciones. Esto implica que, cuando se tienen dos variables relacionadas por una restricción, cada valor posible de una variable debe tener al menos un valor compatible en la otra. En otras palabras, no deberían existir valores en el dominio de una variable que, al ser elegidos, impidan encontrar valores válidos en la otra variable según la restricción que las une.


# Task 2 - CSP con Backtracking, Beam y Local Search


### 1. Definición del problema, variables y restricciones

Las variables que se utilizarán son:

- Exam1
- Exam2
- Exam3
- Exam4
- Exam5
- Exam6
- Exam7

Los dominios de cada variable son los 3 días que se nos indicaron:

- Lunes
- Martes
- Miércoles

Restricciones

En la forma que tomaremos la ejecución de cada examen es la siguiente:

- Rodri tomará Exam1, Exam2 y Exam3
- Suarino tomará Exam2, Exam4 y Exam5
- Sergio tomará Exam3, Exam5 y Exam6
- Andre tomará Exam1, Exam4 y Exam7


In [ ]:
import time

# Definición de variables y dominios
exams = ["Exam1", "Exam2", "Exam3", "Exam4", "Exam5", "Exam6", "Exam7"]
days = ["Lunes", "Martes", "Miércoles"]

# Definición de los estudiantes inscritos en cada examen.
enrollment = {
    "Exam1": ["Rodri", "Andre"],
    "Exam2": ["Rodri", "Suarino"],
    "Exam3": ["Rodri", "Sergio"],
    "Exam4": ["Suarino", "Andre"],
    "Exam5": ["Suarino", "Sergio"],
    "Exam6": ["Sergio"],
    "Exam7": ["Andre"],
}


# Función para contar conflictos en una asignación completa
def count_conflicts(assignment):
    conflicts = 0
    # Se comparan pares de exámenes
    for i in range(len(exams)):
        for j in range(i + 1, len(exams)):
            exam1 = exams[i]
            exam2 = exams[j]
            if exam1 in assignment and exam2 in assignment:
                if assignment[exam1] == assignment[exam2]:
                    # Si comparten estudiante y están en el mismo día, hay conflicto.
                    if set(enrollment[exam1]).intersection(enrollment[exam2]):
                        conflicts += 1
    return conflicts


# Función para verificar si agregar (exam, day) es consistente con la asignación actual.
def is_consistent(assignment, exam, day):
    for other_exam, assigned_day in assignment.items():
        if assigned_day == day:
            # Si exam y otro examen comparten estudiante, no pueden ser el mismo día.
            if set(enrollment[exam]).intersection(enrollment[other_exam]):
                return False
    return True

# Referencias:

- GeeksforGeeks. (2024, 12 junio). Explain the Concept of Backtracking Search and Its Role in Finding Solutions to CSPs. GeeksforGeeks. https://www.geeksforgeeks.org/explain-the-concept-of-backtracking-search-and-its-role-in-finding-solutions-to-csps/

- Boris. (2025, 28 febrero). ARC consistency explained. BorisTheBrave.Com. https://www.boristhebrave.com/2021/08/30/arc-consistency-explained/

- AC-3 algorithm and backtracking. (s. f.). Stack Overflow. https://stackoverflow.com/questions/15462253/ac-3-algorithm-and-backtracking

- Lecture 3 - CSCI E-80. (s. f.). https://cs50.harvard.edu/extension/ai/2021/fall/notes/3/

- Arc consistency. (2005). https://www.cs.cmu.edu/afs/cs/project/jair/pub/volume24/samaras05a-html/node4.html

- Bowen, E. (2024, 19 noviembre). Understanding the beam search algorithm in NLP. Telnyx. https://telnyx.com/learn-ai/beam-search-algorithm

- Achury, N. F. C. (2024, 1 febrero). ¿Qué es y para qué sirven los algoritmos Backtracking o Retroceso?\*. Datum Digital Marketing. https://datumdigitalmarketing.com/noticias/que-es-y-para-que-sirven-los-algoritmos-backtracking-o-retroceso

- Building, C. (s. f.). Local search. https://www.complexica.com/narrow-ai-glossary/local-search
